In [1]:
using LinearAlgebra
using Plots
import StatsBase
include("src/Trajectories.jl")

Main.QuantumTrajectories

In [2]:
using .QuantumTrajectories

In [4]:
?precompute!

search: precompute! precompile permute! __precompile__ promote



No documentation found for public symbol.

`Main.QuantumTrajectories.precompute!` is a `Function`.

```
# 1 method for generic function "precompute!" from Main.QuantumTrajectories:
 [1] precompute!(J::Matrix{ComplexF64}, Heff::Matrix{ComplexF64}, nsamples::Int64, multiplier::Float64, ts::Vector{Float64}, Qs::Vector{Matrix{ComplexF64}})
     @ ~/Documents/Research/MonitoringMetrology/GillipsieAlgorithm/src/functions.jl:7
```


In [6]:
################### Qubit decay example ######################
#### 1. Create the system instance
gamma = 1
sigma_m = [[0.0+0im, 0]  [1, 0]]
sys = System( zeros(ComplexF64, 2, 2), # Hamiltonian
    [sqrt(gamma)*sigma_m ]) #Jump Operators
#### 2. Create the simulation parameters instance
psi0 = zeros(ComplexF64, 2)
psi0[2] = 1 # Initial condition
simulparams = SimulParameters(psi0,
    10.0, # Final time. Set very long so that all trajectories jump
    4, # seed
    100, # Number of trajectories
    10_000, # Number of samples in the finegrid
    1, # Multiplier to use in the fine grid
    1e-3 # Tolerance for passing WTD normalziation
) 
### 3. Run a single trajectory
#data = run_single_trajectory(sys, simulparams)

SimulParameters(psi0=[0.0, 1.0]
nsamples=10000
seed=4
ntraj=100)
multiplier=1.0
tf=10.0
dt=0.001
eps=0.001)

In [8]:
# Stuff for the precomputing 
Qs = Vector{Matrix{ComplexF64}}(undef, simulparams.nsamples)
ts = LinRange(0, simulparams.tf, simulparams.nsamples)

10000-element LinRange{Float64, Int64}:
 0.0, 0.0010001, 0.0020002, 0.0030003, …, 9.996, 9.997, 9.998, 9.999, 10.0

In [9]:
precompute!(sys.J, sys.Heff, 
    simulparams.nsamples, simulparams.multiplier, 
    ts, Qs)

LoadError: MethodError: no method matching precompute!(::Matrix{ComplexF64}, ::Matrix{ComplexF64}, ::Int64, ::Float64, ::LinRange{Float64, Int64}, ::Vector{Matrix{ComplexF64}})
The function `precompute!` exists, but no method is defined for this combination of argument types.

[0mClosest candidates are:
[0m  precompute!(::Matrix{ComplexF64}, ::Matrix{ComplexF64}, ::Int64, ::Float64, [91m::Vector{Float64}[39m, ::Vector{Matrix{ComplexF64}})
[0m[90m   @[39m [35mMain.QuantumTrajectories[39m [90m~/Documents/Research/MonitoringMetrology/GillipsieAlgorithm/src/[39m[90m[4mfunctions.jl:7[24m[39m


In [ ]:
# 